# Day 13: Gradient Boosting - Performance with Discipline (and Leakage Avoidance)

**MGMT 47400 - Predictive Analytics**  
**4-Week Online Course**  
**Day 13 - Thursday June 3, 2027**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/13_gradient_boosting.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain boosting vs bagging at a high level
2. Train a gradient boosting model with sensible defaults
3. Tune learning rate / depth / estimators with runtime controls
4. Compare boosted model vs forest under consistent CV
5. Identify and control overfitting in boosting

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import randint, uniform
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

## 1. Boosting vs Bagging

### Bagging (Random Forest)
- **Strategy**: Train many trees independently on bootstrap samples
- **Combine**: Average (or vote) predictions
- **Effect**: Reduces variance
- **Trees**: Can be deep (low bias, high variance)
- **Parallelizable**: Yes

### Boosting (Gradient Boosting)
- **Strategy**: Train trees sequentially, each correcting previous errors
- **Combine**: Weighted sum of predictions
- **Effect**: Reduces bias (and some variance)
- **Trees**: Should be shallow (reduce variance of individual trees)
- **Parallelizable**: Not really (sequential by nature)

### Gradient Boosting Algorithm (Simplified)

```
1. Start with initial prediction (e.g., mean)
2. For iteration m = 1 to M:
   a. Calculate residuals (errors) from current model
   b. Fit shallow tree to predict residuals
   c. Add tree to model with learning rate η:
      F_m(x) = F_(m-1)(x) + η * tree_m(x)
3. Final model: F_M(x) = sum of all trees
```

**Key insight**: Each tree tries to fix mistakes of previous trees

In [ ]:
# Load data
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")

## 2. Baseline Gradient Boosting

In [ ]:
# Simple GBM with defaults
gbm = GradientBoostingClassifier(random_state=RANDOM_SEED)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
gbm_scores = cross_val_score(gbm, X_train, y_train, cv=cv, scoring='roc_auc')

print("=== GRADIENT BOOSTING (defaults) ===")
print(f"Default params:")
print(f"  n_estimators: 100")
print(f"  learning_rate: 0.1")
print(f"  max_depth: 3")
print(f"\nCV ROC-AUC: {gbm_scores.mean():.4f} ± {gbm_scores.std():.4f}")

# Compare to Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, n_jobs=-1)
rf_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc')

print(f"\n=== RANDOM FOREST (100 trees) ===")
print(f"CV ROC-AUC: {rf_scores.mean():.4f} ± {rf_scores.std():.4f}")

print(f"\n=== COMPARISON ===")
print(f"GBM advantage: {(gbm_scores.mean() - rf_scores.mean()):.4f}")
print("\n💡 GBM often outperforms RF (but needs more tuning)")

## 3. Learning Rate Tradeoff

### Learning Rate (η, or `learning_rate`)

**High learning rate (e.g., 0.3)**
- ✓ Faster training (need fewer trees)
- ✗ Risk of overfitting
- ✗ May miss optimal solution

**Low learning rate (e.g., 0.01)**
- ✓ Better final performance (usually)
- ✓ More stable
- ✗ Needs many more trees
- ✗ Slower training

**Rule of thumb**: Lower learning rate + more trees = better results (but longer training)

In [ ]:
# Learning rate sweep
learning_rates = [0.01, 0.05, 0.1, 0.2, 0.5]
lr_results = []

for lr in learning_rates:
    gbm_lr = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=lr,
        random_state=RANDOM_SEED
    )
    scores = cross_val_score(gbm_lr, X_train, y_train, cv=cv, scoring='roc_auc')
    lr_results.append({
        'learning_rate': lr,
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

lr_df = pd.DataFrame(lr_results)
print("=== LEARNING RATE SWEEP (100 trees) ===")
print(lr_df.to_string(index=False))

# Plot
plt.figure(figsize=(10, 6))
plt.errorbar(lr_df['learning_rate'], lr_df['cv_mean'], 
             yerr=lr_df['cv_std'], marker='o', capsize=5, linewidth=2)
plt.xlabel('Learning Rate')
plt.ylabel('CV ROC-AUC')
plt.title('Effect of Learning Rate (100 trees)')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Lower learning rate often performs better")
print("💡 But needs to compensate with more trees")

## 4. Number of Trees vs Learning Rate

In [ ]:
# Test: low LR with more trees
configs = [
    {'n_estimators': 50, 'learning_rate': 0.1},
    {'n_estimators': 100, 'learning_rate': 0.1},
    {'n_estimators': 200, 'learning_rate': 0.05},
    {'n_estimators': 500, 'learning_rate': 0.01},
]

config_results = []
for config in configs:
    gbm = GradientBoostingClassifier(random_state=RANDOM_SEED, **config)
    scores = cross_val_score(gbm, X_train, y_train, cv=cv, scoring='roc_auc')
    config_results.append({
        'n_estimators': config['n_estimators'],
        'learning_rate': config['learning_rate'],
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

config_df = pd.DataFrame(config_results)
print("=== LEARNING RATE + N_ESTIMATORS COMBINATIONS ===")
print(config_df.to_string(index=False))

print("\n💡 More trees with lower learning rate often wins")
print("💡 But training time increases significantly")

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Train baseline GBM and compare against RF under CV.

Already done above! Now analyze:

---

### YOUR ANALYSIS:

**Performance Comparison:**  
[Which performs better: GBM or RF? By how much?]

**Training Time:**  
[Estimate: which is faster to train?]

**When to use each:**  
[When would you choose GBM over RF?]

---

## 5. Tuning Gradient Boosting

### Key Hyperparameters

1. **n_estimators**: Number of boosting stages
2. **learning_rate**: Shrinkage parameter (η)
3. **max_depth**: Maximum tree depth (keep low: 3-5)
4. **min_samples_split**: Minimum samples to split
5. **subsample**: Fraction of samples for fitting each tree (stochastic GB)

### Tuning Strategy

**Use RandomizedSearchCV with constraints:**
- Limit search space to reasonable values
- Use early stopping to save time
- Monitor overfitting carefully

In [ ]:
# Constrained randomized search
param_distributions = {
    'n_estimators': randint(100, 300),
    'learning_rate': uniform(0.01, 0.19),  # 0.01 to 0.2
    'max_depth': randint(3, 7),
    'min_samples_split': randint(2, 20),
    'subsample': uniform(0.7, 0.3)  # 0.7 to 1.0
}

random_search = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_SEED),
    param_distributions,
    n_iter=20,  # Try 20 combinations
    cv=cv,
    scoring='roc_auc',
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbose=0
)

print("=== RANDOMIZED SEARCH (20 iterations) ===")
print("Running...")
random_search.fit(X_train, y_train)

print(f"\nBest parameters: {random_search.best_params_}")
print(f"Best CV score: {random_search.best_score_:.4f}")

# Test performance
test_score = roc_auc_score(y_test, random_search.predict_proba(X_test)[:, 1])
print(f"Test score: {test_score:.4f}")

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Run constrained tuning and report best params + score.

Already done! Now document your tuning strategy:

---

### YOUR TUNING STRATEGY:

**Best Parameters Found:**  
[List them]

**Performance Improvement:**  
[Compare to baseline GBM]

**What would you try next:**  
[If you had more time/compute, what would you tune?]

---

## 6. Overfitting in Boosting

In [ ]:
# Track train vs validation performance over iterations
from sklearn.model_selection import train_test_split as split2

X_t, X_v, y_t, y_v = split2(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED, stratify=y_train)

gbm_monitor = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    random_state=RANDOM_SEED
)

gbm_monitor.fit(X_t, y_t)

# Compute staged predictions
train_scores = []
val_scores = []

for i, (train_pred, val_pred) in enumerate(zip(
    gbm_monitor.staged_predict_proba(X_t),
    gbm_monitor.staged_predict_proba(X_v)
)):
    train_scores.append(roc_auc_score(y_t, train_pred[:, 1]))
    val_scores.append(roc_auc_score(y_v, val_pred[:, 1]))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_scores)+1), train_scores, label='Train', linewidth=2)
plt.plot(range(1, len(val_scores)+1), val_scores, label='Validation', linewidth=2)
plt.xlabel('Number of Trees')
plt.ylabel('ROC-AUC')
plt.title('Training Progress: Monitoring Overfitting')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find optimal number of trees
optimal_n = np.argmax(val_scores) + 1
print(f"\n=== OPTIMAL NUMBER OF TREES ===")
print(f"Optimal n_estimators: {optimal_n}")
print(f"Validation ROC-AUC: {val_scores[optimal_n-1]:.4f}")
print(f"\n💡 Validation performance peaks, then plateaus (slight overfit after ~{optimal_n} trees)")
print(f"💡 Could use early stopping in production")

## 7. Final Comparison: All Models

In [ ]:
# Final comparison
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

final_models = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
    ]),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=RANDOM_SEED),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=RANDOM_SEED, n_jobs=-1),
    'Gradient Boosting (default)': GradientBoostingClassifier(random_state=RANDOM_SEED),
    'Gradient Boosting (tuned)': random_search.best_estimator_
}

final_results = []
for name, model in final_models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    
    # Test score
    if not hasattr(model, 'predict_proba'):
        model.fit(X_train, y_train)
    if hasattr(model, 'predict_proba'):
        if name not in ['Gradient Boosting (tuned)']:  # Already fitted
            model.fit(X_train, y_train)
        test_pred = model.predict_proba(X_test)[:, 1]
    else:
        test_pred = model.predict(X_test)
    
    test_sc = roc_auc_score(y_test, test_pred)
    
    final_results.append({
        'Model': name,
        'CV_Mean': scores.mean(),
        'CV_Std': scores.std(),
        'Test_Score': test_sc
    })

final_df = pd.DataFrame(final_results).sort_values('CV_Mean', ascending=False)
print("=== FINAL MODEL COMPARISON ===")
print(final_df.to_string(index=False))

print(f"\n✓ Champion: {final_df.iloc[0]['Model']} (CV: {final_df.iloc[0]['CV_Mean']:.4f})")

## 8. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Boosting vs Bagging**: Sequential vs parallel, bias vs variance reduction
2. **Gradient Boosting**: Each tree corrects previous errors
3. **Learning Rate**: Lower rates + more trees = better performance
4. **Tuning Strategy**: Use constrained search, monitor overfitting
5. **Performance**: GBM often beats RF, but needs more careful tuning

### Critical Rules:

> **"Lower learning rate + more trees = better (but slower)"**

> **"Keep trees shallow (depth 3-5) for boosting"**

> **"Monitor overfitting closely with boosting"**

### Next Steps:

- Day 14: Model selection and comparison protocol
- We'll formalize how to choose between models
- Create reproducible experiment logs

---

## Bibliography

- Friedman, J. H. (2001). "Greedy Function Approximation: A Gradient Boosting Machine." *Annals of Statistics*.
- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Tree-Based Methods (boosting overview)
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - Boosting methods
- scikit-learn User Guide: [Gradient boosting](https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting)

---

**End of Day 13 Notebook**